<a href="https://colab.research.google.com/github/AlexFly666/002-openai-quickstart-jike-peng/blob/main/openai_api/09-whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 快速入门 语音识别模型 Whisper

OpenAI 提供了两个基于开源的 Whisper large-v2 模型的语音到文本API服务：
- 转录（transcriptions）：将音频转录为音频所使用的任何语言。
- 翻译（translations）：将音频翻译并转录为英语


目前文件上传限制为 25 MB，支持以下输入文件类型：`mp3、mp4、mpeg、mpga、m4a、wav 和 webm`。


## 语音转录 Transcription API

输入音频文件，返回转录对象（JSON）

参数
- **file**（文件）：需要转录的音频文件对象（不是文件名），支持以下格式：flac、mp3、mp4、mpeg、mpga、m4a、ogg、wav 或 webm。
- **model**（'whisper-1'）：使用的模型 ID。目前仅可使用由我们的开源 Whisper V2 模型驱动的 whisper-1。
- **language**（语言，可选）：输入音频的语言。提供 ISO-639-1 格式的输入语言可以提高准确性和响应速度。
- **prompt**（提示，可选）：可选文本，用于指导模型的风格或继续前一个音频片段。提示应与音频语言相匹配。
- **response_format**（响应格式，可选）：转录输出的格式，默认为 json。可选的格式有：json、text、srt、verbose_json 或 vtt。
- **temperature**（温度，可选）：采样温度，范围从 0 到 1。更高的值，如 0.8，将使输出更随机，而更低的值，如 0.2，将使输出更集中和确定。如果设置为 0，模型将使用对数概率自动提高温度，直到达到某些阈值。
- **timestamp_granularities[]**（时间戳粒度，可选）：为此转录填充的时间戳粒度，默认为 segment。响应格式必须设置为 verbose_json 才能使用时间戳粒度。支持以下一个或两个选项：word 或 segment。注意：segment 时间戳不增加额外延迟，但生成 word 时间戳会增加额外延迟。

返回值
- 转录对象（Transcription Object）或详细转录对象（Verbose Transcription Object）。

### 转录对象

**Transcription Object**:
```json
{
  "text": "Imagine the wildest idea that you've ever had, and you're curious about how it might scale to something that's a 100, a 1,000 times bigger. This is a place where you can get to do that."
}
```

**Verbose Transcription Object**:
```json
{
  "task": "transcribe",
  "language": "english",
  "duration": 8.470000267028809,
  "text": "The beach was a popular spot on a hot summer day. People were swimming in the ocean, building sandcastles, and playing beach volleyball.",
  "segments": [
    {
      "id": 0,
      "seek": 0,
      "start": 0.0,
      "end": 3.319999933242798,
      "text": " The beach was a popular spot on a hot summer day.",
      "tokens": [
        50364, 440, 7534, 390, 257, 3743, 4008, 322, 257, 2368, 4266, 786, 13, 50530
      ],
      "temperature": 0.0,
      "avg_logprob": -0.2860786020755768,
      "compression_ratio": 1.2363636493682861,
      "no_speech_prob": 0.00985979475080967
    },
    ...
  ]
}

```

### 使用 Whisper 实现中文转录

将 TTS 配音的李云龙台词音频文件(liyunlong.mp3)发送给 Whisper 模型进行中文转录

In [5]:
from openai import OpenAI
# client = OpenAI()
# OpenAI Python SDK v1.0 更新后的使用方式
# 使用代理方式，需要修改base_url
client = OpenAI(
    api_key="XXX", # 你的KEY
    base_url="https://vip.apiyi.com/v1"
)
speech_file_path = "./audio/liyunlong.mp3"

# 官方示例的用法会触发 Deprecated 警告⚠️，已替换为最佳实践
with client.audio.speech.with_streaming_response.create(
    model="tts-1",
    voice="echo",
    input="二营长！你他娘的意大利炮呢？给我拉来！"
) as response:
    response.stream_to_file(speech_file_path)

In [6]:
from openai import OpenAI
# client = OpenAI()
# OpenAI Python SDK v1.0 更新后的使用方式
# 使用代理方式，需要修改base_url
client = OpenAI(
    api_key="XXX", # 你的KEY
    base_url="https://vip.apiyi.com/v1"
)
# !mkdir -p ./audio
# !wget https://raw.githubusercontent.com/AlexFly666/002-openai-quickstart-jike-peng/main/openai_api/audio/liyunlong.mp3 -O ./audio/liyunlong.mp3
audio_file= open("./audio/liyunlong.mp3", "rb")

transcription = client.audio.transcriptions.create(
  model="whisper-1",
  file=audio_file
)

print(transcription.text)

二营长,你太娘的义大利泡呢?给我拉来!


In [8]:
# 错误信息 BadRequestError: Error code: 400 - {'error': {'message': "Invalid file format. Supported formats: ['flac', 'm4a', 'mp3', 'mp4', 'mpeg', 'mpga', 'oga', 'ogg', 'wav', 'webm'] (request id: 20250105211719248041153V9Em1JLA)", 'localized_message': '', 'type': 'invalid_request_error', 'param': '', 'code': None}} 明确表明您尝试转录的音频文件的格式不受 OpenAI API 支持。

# 尽管您打开的文件 liyunlong.mp3 看似具有 .mp3 扩展名，但实际文件内容可能不是有效的 MP3 文件。这可能是由于：

# 文件损坏： 文件可能在下载或存储过程中已损坏。
# 文件扩展名不正确： 文件可能被错误地赋予了 .mp3 扩展名，而实际上它是另一种格式。
# 文件编码问题： 文件可能存在编码问题，导致 API 无法将其识别为有效的 MP3 文件。
# 建议的更改
# 要诊断和解决问题，您可以尝试以下操作：

# 验证文件格式： 在 Linux 上使用 file 等工具检查实际文件类型。

# !file ./audio/liyunlong.mp3

!mkdir -p ./audio
# -o 用于指定日志输出文件。
# -O 用于指定下载文件的输出路径和文件名。
!wget https://raw.githubusercontent.com/AlexFly666/002-openai-quickstart-jike-peng/main/openai_api/audio/liyunlong.mp3 -O ./audio/liyunlong.mp3
!file ./audio/liyunlong.mp3

--2025-01-05 13:26:08--  https://raw.githubusercontent.com/AlexFly666/002-openai-quickstart-jike-peng/main/openai_api/audio/liyunlong.mp3
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 77280 (75K) [audio/mpeg]
Saving to: ‘./audio/liyunlong.mp3’

./audio/liyunlong.m 100%[===================>]  75.47K  --.-KB/s    in 0.003s  

2025-01-05 13:26:08 (21.8 MB/s) - ‘./audio/liyunlong.mp3’ saved [77280/77280]

./audio/liyunlong.mp3: MPEG ADTS, layer III, v2, 160 kbps, 24 kHz, Monaural


## 语音翻译 API

输入音频文件，返回翻译文本

请求体
- **file**（文件）：需要翻译的音频文件对象（不是文件名），支持以下格式：flac、mp3、mp4、mpeg、mpga、m4a、ogg、wav 或 webm。
- **model**（'whisper-1'）：使用的模型 ID。目前只有由我们的开源 Whisper V2 模型驱动的 whisper-1 可用。
- **prompt**（提示，可选）：可选文本，用于指导模型的风格或继续前一个音频片段。提示应为英文。
- **response_format**（响应格式，可选）：转录输出的格式，默认为 json。可选的格式包括：json、text、srt、verbose_json 或 vtt。
- **temperature**（温度，可选）：采样温度，范围从 0 到 1。较高的值，如 0.8，将使输出更随机，而较低的值，如 0.2，将使输出更集中和确定。如果设置为 0，模型将使用对数概率自动增加温度，直到达到特定阈值。

返回值
- **translated_text**: 翻译后的文本。

### 使用 Whisper 实现中文识别+翻译

将 TTS 配音的李云龙台词音频文件(liyunlong.mp3)发送给 Whisper 模型进行翻译

In [ ]:
audio_file= open("./audio/liyunlong.mp3", "rb")

translation = client.audio.translations.create(
    model="whisper-1",
    file=audio_file,
    prompt="Translate into English",
)

print(translation.text)

Second Battalion Commander, where is your Italian gun? Bring it to me.


### 使用 TTS 给李云龙英文版台词配音

In [ ]:
speech_file_path = "./audio/liyunlong_en.mp3"

with client.audio.speech.with_streaming_response.create(
    model="tts-1",
    voice="onyx",
    input=translation.text
) as response:
    response.stream_to_file(speech_file_path)

### 使用 Whipser + TTS 生成郭德纲相声英文版

In [ ]:
gdg_audio_file = open("./audio/gdg.mp3", "rb")
gdg_speech_file = "./audio/gdg_en.mp3"

translation = client.audio.translations.create(
  model="whisper-1",
  file=gdg_audio_file
)

print(translation.text)

with client.audio.speech.with_streaming_response.create(
    model="tts-1",
    voice="onyx",
    input=translation.text
) as response:
    response.stream_to_file(gdg_speech_file)

There are a lot of people here. Thank you for coming. Today is the Xiangsheng Conference. Today, He Yunwei and I, the two of us, have a special feature. We don't have much ability, but we give a lot. There are not many opportunities to cooperate with Mr. Xing. Give a little. This is also an old senior in our Xiangsheng world. Mr. Xing Wenzhao has been in the Xiangsheng circle for more than 50 years. That's right. We are going to hold a commemorative exhibition this year. Mr. Xing Wenzhao's third anniversary. 53rd anniversary. That's the 40th anniversary. Oh my god.
